In [1]:
import tensorflow as tf
import numpy as np
import random
import scipy.misc
from logging import getLogger
import datetime
import dateutil.tz
from datetime import date

import os
import sys
import urllib
import pprint
import tarfile

import scipy.misc

In [2]:
# Load MNIST Data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels
valX, valY = mnist.validation.images, mnist.validation.labels

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
class Model:
    def __init__(self, sess, num_visible, num_classes, hparams, model_name, data_type, logs_dir):        
        self.sess = sess
        
        self.num_visible = num_visible
        self.num_classes = num_classes        
        self.num_hidden = hparams['num_hidden']
        
        self.hparams = {}
        
        self.hparams['batch_size'] = hparams['batch_size']
        self.hparams['num_epochs'] = hparams['num_epochs']
        self.hparams['learning_rate'] = hparams['learning_rate']
        self.seed = hparams['seed']
        
        self.gen = hparams['gen']
        self.discr = hparams['discr']
        
        self.batch_normalization = 1.0 / batch_size
        
        self.model_name = model_name
        self.data_type = data_type
        
        self.stats = {}
        self.stats['train_batch_acc'] = {}
        self.stats['train_mean_acc'] = np.zeros(self.hparams['num_epochs'])
        self.stats['val_batch_acc'] = {}
        self.stats['val_mean_acc'] = np.zeros(self.hparams['num_epochs'])
        
        self.logs_dir = logs_dir
        
        self.X = None
        self.Y = None
        self.learning_rate = None
        
        self.W = None
        self.U = None
        self.b = None
        self.c = None
        self.d = None
        
        self.d_U = None
        self.d_W = None
        self.d_b = None
        self.d_c = None
        self.d_d = None
        
        self.updates = []
    
    def _build_model(self):
        with tf.variable_scope(self.model_name):            
            self.X, self.Y, self.learning_rate = self._create_placeholders()                        
            
            m = np.max([self.num_hidden, self.num_classes])
            m_sqrt = np.sqrt(m)
            
            self.U, self.W, self.b, self.c, self.d = self._initialize_variables(m_sqrt)                        
            
            self.model = self._construct_model()
            
            self.d_U, self.d_W, self.d_b, self.d_c, self.d_d = self._define_gradients()
            
            self.updates = [self.U.assign_add(self.hparams['learning_rate'] * self.d_U),
                            self.W.assign_add(self.hparams['learning_rate'] * self.d_W),
                            self.b.assign_add(self.hparams['learning_rate'] * self.d_b),
                            self.c.assign_add(self.hparams['learning_rate'] * self.d_c),
                            self.d.assign_add(self.hparams['learning_rate'] * self.d_d)]
            
            self.predicted_y = tf.argmax(self.model['p_y_all_given_x'], 1)
            self.ground_truth = tf.argmax(self.Y, 1)
            
            
            with tf.name_scope('accuracy'):
                with tf.name_scope('correct_prediction'):
                    self.correct_prediction = tf.equal(self.predicted_y, self.ground_truth)     
                with tf.name_scope('accuracy'):
                    self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32),keep_dims=True)                     
            tf.summary.scalar('accuracy', self.accuracy[0])
            
            # Merge all the summaries and write them out to /tmp/mnist_logs (by default)
            self.merged = tf.merge_all_summaries()
            self.train_writer = tf.train.SummaryWriter(self.logs_dir + '/train', sess.graph)
            self.test_writer = tf.train.SummaryWriter(self.logs_dir + '/test')
            
            tf.initialize_all_variables().run()
            
                    
                
            
    def _get_timestamp(self):
        now = datetime.datetime.now(dateutil.tz.tzlocal())
        return now.strftime('%Y_%m_%d_%H_%M_%S')
    
    def gen_image(img, width, height, outfile, img_type='grey'):
        assert len(img) == width * height or len(img) == width * height * 3

        if img_type == 'grey':
            misc.imsave(outfile, img.reshape(width, height))

        elif img_type == 'color':
            misc.imsave(outfile, img.reshape(3, width, height))
    
    def get_samples(self, height, width):
        weights = self.W.eval()
        rows, _ = weights.shape
        samples = np.asarray([np.reshape(weights[i,:], [height, width]) for i in range(rows)])
        
        return samples        
            
    def save_images(self, images, height, width, n_row, n_col, 
                    cmin=0.0, cmax=1.0, directory="./", prefix="sample"):
        images = images.reshape((n_row, n_col, height, width))
        images = images.transpose(1, 2, 0, 3)
        images = images.reshape((height * n_row, width * n_col))
        filename = '%s_%s.jpg' % (prefix, self._get_timestamp())
        scipy.misc.toimage(images, cmin=cmin, cmax=cmax).save(os.path.join(directory, filename))
    
    def save_samples(self, samples_dir, epoch, height, width):
        samples = self.get_samples(width, height)

        self.save_images(samples, height, width, 50, 10, 
                    directory=samples_dir, prefix="epoch_%s" % epoch)
    
    def _evaluate_per_batch(self, sess, data_X, data_Y, with_updates=False):
        if with_updates:
            summary, _, cost = sess.run([self.merged, self.updates, self.accuracy], feed_dict={self.X: data_X, self.Y: data_Y,
                                                                         self.learning_rate: self.hparams['learning_rate']})            
        else:
            summary, cost = sess.run([self.merged, self.accuracy], feed_dict={self.X: data_X, self.Y: data_Y})        
        return summary, cost
    
    def _evaluate_over_batches(self, sess, data, num_batches, epoch, writer, with_updates=False):
        acc_epoch = np.zeros(num_batches)
        for i in range(num_batches):
            data_x, data_y = data.next_batch(self.hparams['batch_size'])
            summary, cost = self._evaluate_per_batch(sess, self._binarise(data_x), data_y, with_updates)
            acc_epoch[i] = cost[0]
            writer.add_summary(summary, (epoch-1) * num_batches + i)
#             if i % 5000 == 0:
#                 print (np.mean(acc_epoch))
        return acc_epoch
        
            
    def train_and_val(self, sess, train_data, val_data, saver, model_dir, samples_dir):
        num_batches = np.shape(train_data.images)[0] / self.hparams['batch_size']
        width = height = np.sqrt(np.shape(train_data.images)[1])
        
        for epoch in range(1, self.hparams['num_epochs'] + 1):     
            # Training part        
            train_acc_epoch = self._evaluate_over_batches(sess, train_data, num_batches, epoch, self.train_writer, with_updates=True)
                
            self.stats['train_batch_acc'][epoch] = train_acc_epoch
            self.stats['train_mean_acc'][epoch] = np.mean(train_acc_epoch)
            
            # Validation part
            val_acc_epoch = self._evaluate_over_batches(sess, val_data, num_batches, epoch, self.train_writer, with_updates=False)
            
            self.stats['val_batch_acc'][epoch] = train_acc_epoch
            self.stats['val_mean_acc'][epoch] = np.mean(val_acc_epoch)
            
            save_path = saver.save(sess, model_dir+self.model_name+"_"+str(date.today())+"_"+str(epoch)+"_"+str(0)+"_model.ckpt")
            print("Model saved in file: %s" % save_path)
            
            print("Epoch number %d" % epoch)
            print("Training average loss: %f" % self.stats['train_mean_acc'][epoch])
            print("Validate average loss: %f" % self.stats['val_mean_acc'][epoch])
            
#             if self.data_type=="images":                
#                 self.save_samples(samples_dir, epoch, height, width)            
                
                                    
    def _construct_model(self):
        model = self._construct_model_internal()
        return model
            
    def _construct_model_internal(self):
        model = {}
        model['Y_all_classes'] = tf.diag(tf.ones(self.num_classes, 1))
        model['U_all_y'] = tf.reshape(tf.matmul(self.U, model['Y_all_classes']), [1, self.num_hidden, self.num_classes])
        model['WX'] , model['O_all'], model['positive_part'], model['p_y_all_given_x'] = self._give_p_all_y_given_x(model['U_all_y'], model['Y_all_classes'])

        # Calculate p(y|x) for concrete x
        # Result : p_y_given_x (batch_dimension x 1)
        model['p_y_given_x'] = tf.reshape(tf.reduce_sum(tf.mul(model['p_y_all_given_x'], self.Y),1), [-1, 1])        

        # Training part

        # Calculate UY
        # Result : U (batch_dimension x num_hidden)
        model['UY']= tf.transpose(tf.matmul(self.U, tf.transpose(self.Y)))
        
        model['O'] = model['WX'] + model['UY']

        # O_sigma: (batch_dimension x num_hidden)
        model['O_sigma'] = tf.sigmoid(model['O'])

        # O_sigma_all_Y : (batch_dimension x num_hidden x num_classes)
        model['O_sigma_all_Y'] = tf.sigmoid(model['O_all'])

        # O_sigma_all_Y_p : (batch_dimension x num_hidden x num_classes)        
        model['O_sigma_all_Y_p'] = tf.mul(model['O_sigma_all_Y'], tf.tile(tf.reshape(model['p_y_all_given_x'], [-1, 1, self.num_classes]), [1, self.num_hidden, 1]))                         
        
        return model
        
    def _define_gradients(self):
        d_U = tf.zeros([self.num_hidden, self.num_classes], dtype=tf.float32)
        d_W = tf.zeros([self.num_hidden, self.num_visible], dtype=tf.float32)
        d_b = tf.zeros([self.num_visible, 1], dtype=tf.float32)
        d_c = tf.zeros([self.num_hidden, 1], dtype=tf.float32)
        d_d = tf.zeros([self.num_classes, 1], dtype=tf.float32)            
        
        if self.model_name == 'grbm' or self.model_name == 'hybrid':
            # Generative gradients
            d_U_gen, d_W_gen, d_b_gen, d_c_gen, d_d_gen = self._calc_generative_grads(self.Y, self.X)
            d_U = d_U + self.gen * d_U_gen
            d_W = d_W + self.gen * d_W_gen
            d_b = d_b + self.gen * d_b_gen
            d_c = d_c + self.gen * d_c_gen
            d_d = d_d + self.gen * d_d_gen
        
        if self.model_name == 'drbm' or self.model_name == 'hybrid':
            # Discriminative gradients        
            # # d_U: (num_hidden x num_classes)
            print(self.model['O_sigma'])
            dU_left = tf.matmul(tf.transpose(self.model['O_sigma']), self.Y)
            dU_right = tf.matmul(tf.transpose(tf.reduce_sum(self.model['O_sigma_all_Y_p'], 2)), self.Y)
            d_U_disc = self.batch_normalization * (dU_left - dU_right)
            d_U_disc = tf.reshape(d_U_disc, [self.num_hidden, self.num_classes])

            # d_W : (num_hidden x num_visible)
            dW_left = tf.matmul(tf.transpose(self.model['O_sigma']), self.X)
            dW_right = tf.matmul(tf.transpose(tf.reduce_sum(self.model['O_sigma_all_Y_p'], 2)), self.X) 
        
            d_W_disc = self.batch_normalization * (dW_left - dW_right)
            d_W_disc = tf.reshape(d_W_disc, [self.num_hidden, self.num_visible])

            # d_c : (num_hidden x 1)
            dc_left = tf.reduce_sum(self.model['O_sigma'], 0)
            dc_right = tf.reduce_sum(tf.reduce_sum(self.model['O_sigma_all_Y_p'], 2), 0)
            d_c_disc = self.batch_normalization * (dc_left - dc_right)
            d_c_disc = tf.reshape(d_c_disc, [self.num_hidden, 1])

            # d_d : (num_classes x 1)
            d_d_disc = self.batch_normalization * (tf.reduce_sum(self.Y - self.model['p_y_all_given_x'], 0))
            d_d_disc = tf.reshape(d_d_disc, [self.num_classes, 1])
            
            d_U = d_U + self.discr * d_U_disc
            d_W = d_W + self.discr * d_W_disc
            d_c = d_c + self.discr * d_c_disc
            d_d = d_d + self.discr * d_d_disc
        
        return d_U, d_W, d_b, d_c, d_d
        
        
    def _give_p_all_y_given_x(self, U_all_y, Y_all_classes):
        # Tensor of shape (None x num_hidden)
        WX = tf.transpose(tf.matmul(self.W, tf.transpose(self.X)))
    
    
        # WX + c for all batches
        # Result : O (batch_dimension x num_hidden)
        O = WX + tf.reshape(self.c, [1, self.num_hidden])
    
        # Resulted O:
        # Result: O (batch_dimension x num_hidden x num_classes)
        O = tf.reshape(O, [-1, self.num_hidden, 1]) + tf.reshape(U_all_y, [1, self.num_hidden, self.num_classes])

        # First term in log p(y|x) which is calculated for each x in the batch
        # Result : first_term (batch_dimension x num_classes)
        first_term = tf.transpose(tf.matmul(Y_all_classes, self.d))
    
        # Second term in log p(y|x) which is calculated for each x in the batch
        # Result : second_term (batch_dimension x num_classes)            
        second_term = tf.reduce_sum(tf.nn.softplus(O), 1)
    
        # Positive part of log p(y|x)  
        # Result: positive_part (batch_dimension x num_classes)
        positive_part = first_term + second_term

        # Use the softmax to calculate the probabilities:
        # Result: p_y_all_given_x (batch_dimension x num_classes)
        p_y_all_given_x = tf.nn.softmax(positive_part)
    
        return WX, O, positive_part, p_y_all_given_x            
        
        
    def _create_placeholders(self):
        X = tf.placeholder(tf.float32, [None, self.num_visible])
        
        Y = tf.placeholder(tf.float32, [None, self.num_classes])
        
        learning_rate = tf.placeholder(tf.float32)
        
        return X, Y, learning_rate
    
    def _initialize_variables(self, m_sqrt):
        U = tf.get_variable('U', [self.num_hidden, self.num_classes], tf.float32, 
                           tf.random_uniform_initializer(minval=-m_sqrt, maxval=m_sqrt, seed=self.seed, dtype=tf.float32), None)
        
        W = tf.get_variable('W', [self.num_hidden, self.num_visible], tf.float32, 
                           tf.random_uniform_initializer(minval=-m_sqrt, maxval=m_sqrt, seed=self.seed, dtype=tf.float32), None)
        
        b = tf.get_variable('b', [self.num_visible, 1], tf.float32,
                           tf.zeros_initializer, None)
        
        c = tf.get_variable('c', [self.num_hidden, 1], tf.float32,
                           tf.zeros_initializer, None)
        
        d = tf.get_variable('d', [self.num_classes, 1], tf.float32,
                           tf.zeros_initializer, None)  
        
        return U, W, b, c, d
    
    def _sample_prob(self, probs, size):    
        rand = tf.random_uniform([1, size], minval=0.0, maxval=1.0, dtype=tf.float32)
        
        return tf.cast(rand < probs, tf.float32)
    
#     def _sample_vec(self, probs, size):
        
        
#         return tf.nn.relu(tf.sign(probs - rand))

    def _calc_generative_grads(self, y, x):
        y0, x0, h0, y1, x1, h1 = self._gibbs_sampling_step(y, x)
        
        d_U_gen = self.batch_normalization * ( tf.transpose(h0) * y0 - tf.transpose(h1) * y1)
        d_W_gen = self.batch_normalization * ( tf.transpose(h0) * x0 - tf.transpose(h1) * x1)
        
        d_b_gen = self.batch_normalization * (tf.reduce_sum(x0 - x1, 0))
        d_b_gen = tf.reshape(d_b_gen, [self.num_visible, 1])
        
        d_c_gen = self.batch_normalization * (tf.reduce_sum(h0 - h1,0))
        d_c_gen = tf.reshape(d_c_gen, [self.num_hidden, 1])
        
        d_d_gen = self.batch_normalization * (tf.reduce_sum(y0 - y1, 0))
        d_d_gen = tf.reshape(d_d_gen, [self.num_classes, 1])
        
        return d_U_gen, d_W_gen, d_b_gen, d_c_gen, d_d_gen
        
    
    def _gibbs_sampling_step(self, y, x):
        # Positive phase
        y0 = y
        x0 = x
        h0 = tf.transpose(tf.nn.sigmoid(self.c + tf.matmul(self.W,tf.transpose(x0)) + tf.matmul(self.U, tf.transpose(y0))))        
    
        # Negative phase
        h0new = self._sample_h(h0)    
        y1 = self._sample_y(h0new)
        x1 = self._sample_x(h0new)
        h1 = tf.transpose(tf.nn.sigmoid(self.c + tf.matmul(self.W, tf.transpose(x1)) + tf.matmul(self.U, tf.transpose(y1))))
    
        return y0, x0, h0, y1, x1, h1

    def _sample_h(self, h_prob):            
#         return self._binarise(h_prob)
        return self._sample_prob(h_prob, self.num_hidden)
    def _sample_y(self, h):
        yprob = tf.transpose(tf.nn.softmax(self.d + tf.matmul(tf.transpose(self.U), tf.transpose(h))))
        
#         y_sampled = tf.cast(tf.multinomial(yprob, 1), tf.int32)

#         u = tf.random_uniform([self.hparams['batch_size']])
        
# #         u = tf.random_uniform([1, self.num_classes], minval=0.0, maxval=1.0, dtype=tf.float32)
#         less_eq = tf.cast((u <= tf.cumsum(yprob, 1)), tf.int32)
#         sample = tf.cast(tf.equal(less_eq, tf.ones(self.num_classes, tf.int32)), tf.float32)
        
#         return sample
        return tf.squeeze(tf.gather(self.model['Y_all_classes'], tf.cast(tf.multinomial(yprob, 1), tf.int32)), [1])
        
        
        
        
        
        

        
#         print(tf.tile(tf.reshape(tf.range(self.num_classes),[1,self.num_classes]),[-1,1]) == tf.tile(y_sampled, [-1,self.num_classes]))
        
        
#         all_batches_y = tf.tile(tf.reshape(self.model['Y_all_classes'],[1, self.num_classes, self.num_classes]),[-1, 1, 1])
        
        
        
#         print(tf.tile(tf.reshape(tf.range(self.num_classes),[1,self.num_classes]),[-1,1]) == y_sampled)

#         return self.model['Y_all_classes'][tf.cast(tf.multinomial(yprob, 1), tf.int32)]
#         return sefl._binarise(yprob)
#         return self._sample_prob(yprob, self.num_classes)
    def _sample_x(self, h):
        xprob = tf.transpose(tf.nn.sigmoid(self.b + tf.matmul(tf.transpose(self.W), tf.transpose(h))))
        return self._sample_prob(xprob, self.num_visible)
    
    def _binarise(self, images):
        return (images > 0).astype('float32')
#         return (np.random.uniform(size=images.shape) < images).astype('float32')    
    def variable_summaries(var):
        """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
        with tf.name_scope('summaries'):
            mean = tf.reduce_mean(var)
            tf.summary.scalar('mean', mean)
            with tf.name_scope('stddev'):
                stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
                tf.summary.scalar('stddev', stddev)
                tf.summary.scalar('max', tf.reduce_max(var))
                tf.summary.scalar('min', tf.reduce_min(var))
                tf.summary.histogram('histogram', var)
    

        
        

In [4]:
# Model HyperParameters
num_visible = 28*28
num_classes = 10
num_hidden = 500
batch_size = 1
num_epochs = 100
num_batches = np.shape(trX)[0] / batch_size

if np.shape(trX)[0] % batch_size > 0:
    num_batches += 1

In [5]:
config = tf.ConfigProto(allow_soft_placement = True)

In [ ]:
hparams = {}
hparams['num_hidden'] = num_hidden
hparams['batch_size'] = batch_size
hparams['num_epochs'] = num_epochs
hparams['learning_rate'] = 0.05
hparams['seed'] = 123
hparams['gen'] = 0.01
hparams['discr'] = 1.0
model_names= {'drbm', 'grbm', 'hybrid'}

In [ ]:
with tf.Session(config = config) as sess:
    model = Model(sess, num_visible, num_classes, hparams, model_name='drbm', data_type='images', logs_dir='./logs')
    model._build_model()
#     sess.run(tf.initialize_all_variables())
    model.train_and_val(sess, mnist.train, mnist.validation, tf.train.Saver(), './models/', './samples/')
    model.train(sess, mnist.train)

Tensor("drbm/Sigmoid:0", shape=(?, 500), dtype=float32)


In [ ]:
# Testing of all the models :P
